<div><img src="Images/SKI_Header_1_Tr.png" width="150%"/></div>
<p style="text-align: center; font-size: 36pt; font-weight: bold; color: rgb(0, 0, 190); padding: 20px; margin-top: 2%;">КУРС</p>
<p style="text-align: center; font-size: 36pt; font-weight: bold; color: rgb(0, 0, 190); padding: 40px; margin-top: -3%; line-height: 100%;">Архитектура ЭВМ и язык ассемблера</p>
<p style="text-align: center; font-size: 28pt; font-weight: bold; color: rgb(0, 0, 190); padding: 40px; margin-top: -3%; line-height: 100%;">СЕМИНАРЫ</p>
<p style="text-align: left; font-size: 20pt; font-weight: bold; color: rgb(0, 0, 190); padding: 20px; margin-top: -2%; line-height: 10%;">Преподаватель:</p>
<p style="text-align: left; font-size: 20pt; font-weight: bold; color: rgb(0, 0, 190); padding: 18px; margin-top: -2%; line-height: 100%;">&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;Русол Андрей Владимирович</p>
<p style="text-align: left; font-size: 20pt; font-weight: bold; color: rgb(0, 0, 190); padding: 18px; margin-top: -2%; line-height: 100%;">&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;к.т.н., с.н.с. ГЕОХИ РАН</p>
<p style="text-align: center; font-size: 16pt; font-weight: bold; color: rgb(0, 0, 190); padding: 20px; margin-top: 2%;">Москва 2022</p>


<p style="text-align: center; font-size: 36pt; font-weight: bold; color: rgb(0, 0, 190); padding: 40px; margin-top: -3%; line-height: 100%;">Семинары № 9 и 10</p>


<p style="text-align: center; font-size: 28pt; font-weight: bold; color: rgb(0, 0, 190); padding: 40px; margin-top: -3%; line-height: 100%;">Подпрограммы в языке Ассемблер</p>

## Оформление подпрограмм

Подпрограмма - это обособленный фрагмент кода, выполняющий определенную задачу, которая может потребоваться в разных частях основной программы. В языке Ассемблер подпрограмма может быть формлена с помощью меток. 

#### Вызов подпрограммы

Вызов подпрограммы выполняется с помощью команды **CALL**, которая работает почти также, как команда безусловного перехода **JMP**, но с важным отличием — одновременно в стек сохраняется текущее значение регистра **IP**. Это позволяет потом вернуться к следующей после вызова подпрограммы команде. В качестве операнда указывается адрес перехода, который может быть непосредственным значением (меткой). 

```asm
call <имя подпрограммы>
```

#### Возврат из подпрограммы

Возврат из подпрограммы выполняется командой **RET**, которая в простейшем варианте не имеет параметров. Эта команда восстанавливает значение из **вершины стека** в регистр **IP**. Таким образом, выполнение программы продолжается с команды, следующей сразу после команды **CALL**. Команды **CALL** и **RET** не изменяют значения флагов. 

![](Images/QWERTY_ASCII.png)

#### Пример

Написать подпрограмму, которая заполняет область памяти различной длины заданным значением.
**edi** - адрес начала области памяти 
**ecx** - длина массива 
**al** - значение заполнения

```asm
%include "io.inc"
section .bss
n equ 5                      
m equ 10                      
arr resb n 
arr2 resb m

section .text
global CMAIN
CMAIN:
    mov ebp, esp; for correct debugging
    mov edi, arr
    mov ecx, n
    mov al, 'A'
    PRINT_DEC 2,al
    PRINT_STRING `\n`    
    call fill_mem

    mov edi, arr2
    mov ecx, m
    mov al, 'B'
    PRINT_DEC 2,al
    PRINT_STRING `\n`
    call fill_mem
                                      
    ret
    
; Подпрограмма fill_mem
fill_mem:
    jecxz .fm_q
.fm_lp: 
    mov [edi], al
    inc edi
    loop .fm_lp
.fm_q:
    ret    
```

Такой способ сработал только из-за того, что передача данных и параметров осуществляется через регистры. В большинстве реальных случаев для обеспечения работы подпрограмм требуется использование стека, т.к. регистров может просто не хватать или значения нужны длительное время, а также через регистры не удастся организовать рекурсивный вызов подпрограммы.

# Стековый фрейм

**Стек** - структура данных, организованная по принципу **LIFO** («Last In — First Out» или «последним пришёл — первым ушёл»). Стек часть архитектуры процессора и поддерживается на аппаратном уровне. В процессорах реализуются специальные регистры **SS, BP, SP** и команды для работы со стеком.

Стек используется для сохранения содержимого регистров, используемых программой, перед вызовом подпрограммы, чтобы подпрограмма могла их использовать для решения поствленных перед ней задач.

Стек располагается в оперативной памяти в специальном сегменте, и адресуется относительно сегментного регистра **SS**. Шириной стека называется размер элементов, которые можно помещать в него или извлекать. Для работы со стеком реализованы две основные операции:
```asm
    PUSH  –  добавление элемента на вершину стека
    POP   –  извлечение элемента с вершины стека
```

##  Добавление элемента в стек

Добавление элемента в стек выполняется командой **PUSH**, у нее один операнд, который может быть непосредственным значением, 16-битным регистром (в том числе сегментым) или 16-битной переменной в памяти. Команда работает следующим образом:

- значение в регистре **SP** уменьшается на 2 (так как ширина стека — 16 бит или 2 байта);
- операнд помещается в память по адресу в **SP**.


##  Извлечение элемента из стека

Извлечение элемента из стека выполняется командой **POP**. У этой команды также один операнд, который может быть 16-битным регистром (в том числе сегментым, но **кроме CS**) или 16-битной переменной в памяти. Команда работает следующим образом:

- операнд читается из памяти по адресу в **SP**;
- значение в регистре **SP** увеличивается на 2.

#### Пример

```asm
%include "io.inc"
section .data
x dw 5
section .text
global CMAIN
CMAIN:
    mov ebp, esp; for correct debugging
    xor eax, eax
    xor ebx, ebx
    push word [x]
    pop bx
    push word 15
    pop bx
    mov ax, 25
    push word ax
    pop bx
    ret
```

## Работа стека при вызве функции

Стек рассчитан на косвенную адресацию через специальный регистр **ЕSP** - указатель стека (Stack Pointer). При помещении элементов в стек происходит **автоматический декремент** указателя стека, а при извлечении - **автоматический инкремент**, таким образом **стек растет в сторону младших адресов**. Адрес последнего включенного в стек элемента, содержащийся в регистре указателя стека, называется вершиной стека. Стек оперирует только **словами**, так что попытка включения байта все равно приведет к включению целого слова.

**Физический адрес стека** формируется из содержимого регистров **ЕSP** и **SS** или **ЕBP** и **SS** , причем **ЕSP** служит неявным указателем стека для всех операций включения и извлечения, а **SS** - сегментным регистром стека. Значение регистра **SS** является самым младшим адресом (т. е. границей) области стека и называется **базой стека**. Начальное значение регистра **ЕSP** есть наибольшее смещение, которого может достигать стек. Регистр **ЕBP** предназначен для произвольных обращений к стеку.

Стек эффективнее обычной памяти в двух отношениях. Команды **PUSH** и **POP** короче команды **MOV**, так как один из операндов косвенно адресуется через регистр **ЕSP** , а инкремент или декремент регистра **ЕSP** с образованием
нового целевого адреса производится автоматически.

При вызове подпрограммы необходимо выполнить следующие три действия:
- в отличие от других команд переходов вызов подпрограммы должен запомнить адрес следующей команды, чтобы можно было осуществить возврат в нужное место вызывающей программы;
- используемые подпрограммой регистры необходимо запомнить до изменения их содержимого, а перед самым выходом из подпрограммы восстановить эти регистры;
- подпрограмма должна иметь средства взаимодействия или разделения данных с вызвавшей ее программой и другими подпрограммами.

Команда **CALL** не только осуществляет переход по указанному адресу, но и помещает в стек адрес возврата (т. е. адрес команды, непосредственно следующей за командой **CALL**). Команда **RET** просто извлекает адрес возврата из стека. При этом предполагается, что если подпрограмма обращалась к стеку, то указатель стека возвращен на прежнее значение и действия со стеком не разрушили адрес возврата.

В команде **RET** допускается необязательный операнд, который прибавляется к содержимому регистра **ЕSP** после извлечения из стека адреса возврата.





Основная программа или вызывающая подпрограмма размещает в стеке параметры, затем при вызове подпрограммы в стек заносится адрес возврата, а после этого вызванная подпрограмма резервирует место в стеке под локальные переменные. Все вместе образует ***стековый фрейм***. Для корректной работы со стековым фреймом необходимо **первым действием подпрограммы** необходимо сохранить значение регистра **ESP** в регистр **EBP** и использовать его для доступа к параметрам и локальным переменным, а **ESP** продолжает работать как указатель стека. Перед **завершением подпрограммы** необходимо восстановить значение **ESP** передав в него значение из **EBP**.

Первые действия
```asm
push ebp
mov ebp, esp
sub esp, <размер локальных данных в байтах>
```

Последние действия
```asm
mov esp, ebp
pop ebp
ret
```


<div><img src="Images/StFr.png" width="35%"/></div>




Таким образом **[ebp - x]** - адреса локальных переменных, **[ebp]** - сохраняемое предыдущее значение **EBP**, **[ebp + 4]** - адрес возврата в вызвавщий фрагиент кода, **[ebp + x]** - параметры функции.





#### Пример

Получаем со стандартного ввода два числа, передаем их в стек, вызываем подпрограмму, которая читает числа из стека и складывает их, размещая результат в регистре **ЕАХ**. 

```asm
%include "io.inc"

section .text
global CMAIN
CMAIN:
    push ebp
    mov ebp, esp
    sub esp,8    
    GET_DEC 4,[esp]
    GET_DEC 4,[esp+4]
    call my_add
    PRINT_DEC 4,eax
    leave    
    ret
    
my_add:
    push ebp
    mov ebp, esp
    mov eax, dword [ebp+8] 
    add eax, dword [ebp+12]
    mov esp, ebp
    pop ebp
    ret
```

Команда **LEAVE** только копирует **EBP** в **ESP**, тем самым выбрасывая весь последний кадр стека, созданный последней выполненной подпрограммой, и считывает из стека **EBP** для вызывавшего кода, что одновременно восстанавливает и значение, которое имел **ESP** до вызова подпрограммы. Результатом выполнения команды **LEAVE** является корректное завершение работы состеком. 

Для обеспечения корректного начала работы со стеком существует команда **ENTER**, которая создаёт стековый кадр заданного размера и уровня вложенности (оба операнда – числа; уровень вложенности может принимать значения только от 0 до 31). 

***ВНИМАНИЕ!*** применение команды **ENTER** не рекомендуется, т.к. обоаботка параметров так и не риализована (вариант **ENTER 0, 0** всегда будет срабатывать штатно), а время ее выполнения существенно (по некоторым данным до 5 раз) превышает время выполнения стандартного пролога
```asm
push ebp
mov ebp, esp
```

Команда  **LEAVE**, наоборот, срабатывает быстрее (насколько данных нет), чем стандартый эпилог
```asm
mov esp, ebp
pop ebp
```

#### Пример

Получаем со стандартного ввода три числа, передаем их в стек, вызываем подпрограмму, которая читает числа из стека и складывает их, размещая результат в регистре **ЕАХ**. 

```asm
%include "io.inc"

section .text
global CMAIN
CMAIN:
    push ebp
    mov ebp, esp
    sub esp,12    
    GET_DEC 4,[esp]
    GET_DEC 4,[esp+4]
    GET_DEC 4,[esp+8]
    call my_add
    PRINT_DEC 4,eax
    mov esp, ebp 
    pop ebp     
    ret
    
my_add:
    enter 0, 0
    mov eax, dword [ebp+8] 
    add eax, dword [ebp+12]
    add eax, dword [ebp+16]
    leave
    ret
```



#### Пример

Передаем переменную в стек, и вызываем стандартную функцию **printf**, которая читает данные с вершины стека и выводит на стандартный вывод. 

```asm
%include "io.inc"

section .data
msg db `Hello, World !!! \n`, 0


section .text
global CMAIN
CMAIN:
    push ebp
    mov ebp, esp
    sub esp,8  
    mov dword [esp], msg 
    call printf
    mov esp, ebp 
    pop ebp     
    ret
```

#### Пример

Это уже знакомый нам пример когда команда короткого перехода **JECXZ** вызывает ошибку из-за макроса печати **PRINT_STRING**

```asm
%include "io.inc"

section .data  
n dd 0
str: db `Hello, World!\n`, 0

section .text
global CMAIN
CMAIN:
    mov ecx, [n]
    jecxz EXIT
CYCL:
    PRINT_STRING [str]  
    dec ecx
    cmp ecx,0
    jne CYCL

EXIT:
    mov eax, 0
    ret
```    

#### Пример

Прграмма, использующая команду короткого перехода **JECXZ** и системые прерывания.
Выполняется без ошибок.
```asm

%include "io.inc"

section .data  
n dd 10
str: db `Hello, World!\n`, 0

section .text
global CMAIN
CMAIN:
    mov ebp, esp; for correct debugging
    mov ecx, [n]
    jecxz EXIT
CYCL:
    push ecx
    mov eax, 0x04   ; записать шестнадцатиричное 4 в регистр eax (0x04 = write())    
    mov ebx, 0x1    ; дескриптор вывода (1 = стандартный вывод)    
    mov ecx, str    ; указатель на выводимую строку
    mov edx, 15     ; длина строки
    int 0x80        ; отправляем сигнал прерывания 0x80, который ОС интерпретирует как системный вызов
    pop ecx                
    dec ecx
    cmp ecx,0
    jne CYCL

EXIT:
    mov eax, 0x01   ; записать шестнадцатиричное 1 в регистр eax (0x01 = exit())
    mov ebx, 0      ; 0 = нет ошибок
    int 0x80
```    
    
    
#### Пример

Получаем со стандартного ввода числo, передаем его в стек, вызываем подпрограмму, которая читает число из стека и вычисляет его факториал, размещая результат в регистре **ЕАХ**. 
```asm
%include "io.inc"

section .text
global CMAIN
CMAIN:
    mov ebp, esp; for correct debugging
    push ebp
    mov ebp, esp
    sub esp,4    
    GET_DEC 4,[esp]
    call factorial
    PRINT_DEC 4,eax
    mov esp, ebp 
    pop ebp     
    ret
    
factorial:
    push ebp
    mov ebp, esp
    sub esp,4    
    mov ecx, dword [ebp+8]
    mov eax, 1 
    cmp ecx, 0
    je .zero
    .cycl:
        imul eax,ecx
        dec ecx
        cmp ecx,0
        jne .cycl
        
    .zero:    
    leave
    ret
```

#### Пример

Получаем со стандартного ввода числo, передаем его в стек, вызываем подпрограмму, которая вычисляет его факториал с помощью рекурсивных вызовов, размещая результат в регистре **ЕАХ**. 
```asm
%include "io.inc"

section .text
global CMAIN
CMAIN:
    mov ebp, esp; for correct debugging
    push ebp
    mov ebp, esp
    sub esp,4    
    GET_DEC 4,[esp]
    mov ecx, dword [esp]    
    call factorial
    PRINT_DEC 4,eax
    mov esp, ebp 
    pop ebp     
    ret
    
factorial:
    cmp     ecx, 1                  ; проверяем n <= 1?
    jnbe    .rec                    ; eсли нет, делаем рекурсивный вызов
    mov     eax, 1                  ; иначе возвращаем 1
    ret
    .rec:
        push    ecx                     ; сохраняем n в стек
        dec     ecx                     ; n-1
        call    factorial               ; вызываем factorial(n-1)
        pop     ecx                     ; восстанавливаем n из стека
        imul    eax, ecx                ; n * factorial(n-1)
        ret 
```

    